In [1]:
# Enable auto-reloading of external modules - useful during development
%load_ext autoreload
%autoreload 2

# Configure Python path to find our custom modules
import sys
from pathlib import Path

# Add project root to the Python path for proper imports
project_root = Path.cwd().parent
if project_root not in sys.path:
    sys.path.insert(0, str(project_root))

In [2]:
# Import necessary libraries
import src.processing as processing
import src.config as lists

In [3]:
# Load and process data
df = processing.load_data("/Users/luis.m/Library/Mobile Documents/com~apple~CloudDocs/Documents ☁️/VSC Projects/Master_Thesis/data/raw/nvzfxcoxdvh1at7i.csv")
df_prepared = processing.prepare_data(df)
df_added_features = processing.create_all_model_features_orchestrated(df_prepared)
df_missing = processing.drop_missing_final_vars_streamlined(df_added_features, lists.final_set_A_predictor_names_and_dependent)
df_final = processing.annual_winsorize_variables(df_missing, lists.columns_to_winsorize)

/Users/luis.m/Library/Mobile Documents/com~apple~CloudDocs/Documents ☁️/VSC Projects/Master_Thesis/src/processing.py:42: DtypeWarning: Columns (10,11,12,16,17,22,26,30,33,34,35,945,946,948,949,950,955,969,970,975,980) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path)


Data loaded successfully from /Users/luis.m/Library/Mobile Documents/com~apple~CloudDocs/Documents ☁️/VSC Projects/Master_Thesis/data/raw/nvzfxcoxdvh1at7i.csv
Original number of observations: 317304
Number of columns after selection: 30
Observations after year filter (2000-2023): 302751
Observations after excluding financial and utility firms: 170598
Starting feature construction. Initial df shape: (170598, 30)
  Creating lags for: ['at', 'ni', 'rect', 'invt', 'ap', 'sale']

Performing pre-calculation validity checks & preparations...
  Missing 'xrd' values filled with 0.
  'ipo_year' created from 'ipodate'.

Constructing dependent variable...
  OCF_Scaled_t_plus_1 created.

Constructing Set A (OLS) predictors...
  Set A predictors constructed.

Constructing control dummy variables...
  Dummy variables constructed.

Constructing Set B (additional ML) predictors...
  Set B predictors constructed.

Selecting final model variables and dropping intermediate columns...
  Shape of DataFrame 

In [4]:
# Split data chronologically - train on pre-2018, test on 2018+
train_df, test_df = processing.split_data_chronologically(df_final, 'fyear', split_year=2018)

print(f"Training data shape: {train_df.shape}")
print(f"Test data shape: {test_df.shape}")
print(f"Training period: {train_df['fyear'].min()} - {train_df['fyear'].max()}")
print(f"Test period: {test_df['fyear'].min()} - {test_df['fyear'].max()}")

Training set: 102567 obs (Predictor years <= 2018)
Test set: 20882 obs (Predictor years > 2018)
Training data shape: (102567, 30)
Test data shape: (20882, 30)
Training period: 2001.0 - 2018.0
Test period: 2019.0 - 2022.0


In [5]:
# Prepare features and dependent variable for XGBoost regression
X_train = train_df[lists.SET_A_FEATURES + lists.CONTROL_DUMMY_FEATURES]
y_train = train_df[lists.DEPENDENT_VARIABLE]

X_test = test_df[lists.SET_A_FEATURES + lists.CONTROL_DUMMY_FEATURES]
y_test = test_df[lists.DEPENDENT_VARIABLE]

# Drop dummy variables that caused multicollinearity in OLS
X_train = X_train.drop(columns=['ASC842_dummy', 'COVID_dummy'])
X_test = X_test.drop(columns=['ASC842_dummy', 'COVID_dummy'])

print("Features included in the model:")
for i, feature in enumerate(X_train.columns, 1):
    print(f"{i:2d}. {feature}")

print(f"\nDependent variable: {lists.DEPENDENT_VARIABLE}")
print(f"Training observations: {len(X_train)}")
print(f"Test observations: {len(X_test)}")

Features included in the model:
 1. OCF_Scaled_Lag_t
 2. NI_Scaled_t
 3. Accruals_Scaled_t
 4. Delta_Rec_Scaled_t
 5. Delta_Inv_Scaled_t
 6. Delta_AP_Scaled_t
 7. DP_Scaled_t
 8. ln_at_t
 9. ASC606_dummy
10. TCJA_dummy

Dependent variable: OCF_Scaled_t_plus_1
Training observations: 102567
Test observations: 20882


In [6]:
# =============================================================================
# CROSS-VALIDATION FOR HYPERPARAMETER TUNING
# =============================================================================

import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import numpy as np
import time

# Define hyperparameter grid for XGBoost
param_grid = {
    'max_depth': [3, 5, 7, 10],
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 200, 300],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0]
}

# Create XGBoost regressor
xgb_regressor = xgb.XGBRegressor(random_state=42, objective='reg:squarederror')

# Perform Grid Search with 5-fold cross-validation
print("Performing 5-fold cross-validation for XGBoost hyperparameter tuning...")
print(f"Total parameter combinations to test: {len(param_grid['max_depth']) * len(param_grid['learning_rate']) * len(param_grid['n_estimators']) * len(param_grid['subsample']) * len(param_grid['colsample_bytree'])}")

start_time = time.time()

grid_search = GridSearchCV(
    estimator=xgb_regressor,
    param_grid=param_grid,
    cv=5,
    scoring='r2',
    n_jobs=-1,
    verbose=2  # Increased verbosity for more detailed progress information
)

# Fit the grid search
grid_search.fit(X_train, y_train)

# Print best parameters and score
print(f"\nBest hyperparameters: {grid_search.best_params_}")
print(f"Best cross-validation R² score: {grid_search.best_score_:.4f}")
print(f"Standard deviation: {grid_search.cv_results_['std_test_score'][grid_search.best_index_]:.4f}")

elapsed_time = time.time() - start_time
print(f"Grid search completed in {elapsed_time:.2f} seconds")

Performing 5-fold cross-validation for XGBoost hyperparameter tuning...
Total parameter combinations to test: 324
Fitting 5 folds for each of 324 candidates, totalling 1620 fits
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.8; total time=   0.4s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.8; total time=   0.4s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.8; total time=   0.4s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.9; total time=   0.4s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.8; total time=   0.4s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.9; total time=   0.4s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.8; total time=   0.5s
[CV] END c

In [7]:
# =============================================================================
# XGBOOST MODEL - For ML Model Comparisons
# =============================================================================

# Use the best model from grid search
best_xgb_model = grid_search.best_estimator_

# Predictions
y_train_pred = best_xgb_model.predict(X_train)
y_test_pred = best_xgb_model.predict(X_test)

# Simple metrics function for any model
def print_model_performance(y_true, y_pred, model_name, dataset):
    r2 = r2_score(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    print(f"{model_name} ({dataset}): R²={r2:.4f} | RMSE={rmse:.4f} | MAE={mae:.4f}")

# Print results
print("XGBOOST PERFORMANCE:")
print_model_performance(y_train, y_train_pred, "XGBoost", "Train")
print_model_performance(y_test, y_test_pred, "XGBoost", "Test")

XGBOOST PERFORMANCE:
XGBoost (Train): R²=0.7426 | RMSE=0.5377 | MAE=0.1733
XGBoost (Test): R²=0.6473 | RMSE=0.5792 | MAE=0.1963


In [8]:
# =============================================================================
# FEATURE IMPORTANCE ANALYSIS
# =============================================================================

import pandas as pd

# Get feature importances
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': best_xgb_model.feature_importances_
}).sort_values('importance', ascending=False)

print("FEATURE IMPORTANCE RANKING:")
print("="*50)
for i, (_, row) in enumerate(feature_importance.iterrows(), 1):
    print(f"{i:2d}. {row['feature']:<25} {row['importance']:.4f}")

# Print model details
print(f"\nMODEL DETAILS:")
print(f"Max Depth: {best_xgb_model.max_depth}")
print(f"Learning Rate: {best_xgb_model.learning_rate}")
print(f"N Estimators: {best_xgb_model.n_estimators}")
print(f"Subsample: {best_xgb_model.subsample}")
print(f"Colsample Bytree: {best_xgb_model.colsample_bytree}")
print(f"Number of Features: {best_xgb_model.n_features_in_}")

FEATURE IMPORTANCE RANKING:
 1. NI_Scaled_t               0.4925
 2. ln_at_t                   0.2025
 3. TCJA_dummy                0.0975
 4. OCF_Scaled_Lag_t          0.0934
 5. ASC606_dummy              0.0428
 6. Accruals_Scaled_t         0.0233
 7. Delta_AP_Scaled_t         0.0201
 8. Delta_Rec_Scaled_t        0.0114
 9. DP_Scaled_t               0.0095
10. Delta_Inv_Scaled_t        0.0070

MODEL DETAILS:
Max Depth: 3
Learning Rate: 0.1
N Estimators: 200
Subsample: 1.0
Colsample Bytree: 0.8
Number of Features: 10
